In [1]:
from torchvision.transforms import v2
from common.model.embedding.predefined.w2vbert import W2VBertFoundationEmbedder
from torchaudio.transforms import Resample
from torch import nn
import common.data.audio.transforms as audtransforms
import common.data.signal.transforms

transform = nn.Sequential(
    audtransforms.AudioSequencePartitioning(
        fs=44100, sequence_duration_seconds=2, channels_first=True,
        resampler=nn.Sequential(
            v2.Lambda(lambda x: x.T),
            Resample(44100, 16000),
            v2.Lambda(lambda x: x.T),
            common.data.signal.transforms.SignalZeroMasking(2, 16000, channels_first=False)),
    ),
    audtransforms.W2VBertFeatureExtractorTransform(force_time_seq=True),
    W2VBertFoundationEmbedder()
)

/home/jacopo/PycharmProjects/progetto-tesi/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
import torch

# Video di 11s
o = torch.randn((17, 352800))

In [3]:
out = transform(o)

It is strongly recommended to pass the `sampling_rate` argument to `SeamlessM4TFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


In [4]:
out.shape

torch.Size([4, 99, 1024])

In [6]:
o1 = torch.randn((17, 3568000))
out1 = transform(o1)
out1.shape

It is strongly recommended to pass the `sampling_rate` argument to `SeamlessM4TFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


torch.Size([41, 99, 1024])

In [85]:
from transformers import AutoProcessor, WavLMModel, AutoFeatureExtractor

processor = AutoFeatureExtractor.from_pretrained("microsoft/wavlm-base")
model = WavLMModel.from_pretrained("microsoft/wavlm-base")

In [90]:
a = torch.randn(16000)

In [91]:
b = processor(a.unbind(0), return_tensors="pt")

It is strongly recommended to pass the `sampling_rate` argument to `Wav2Vec2FeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


In [88]:
b.data["input_values"]

tensor([[[-0.4366,  0.9578, -0.2994,  ...,  0.7621, -0.8440,  0.1853],
         [ 0.9509, -1.9667, -1.1055,  ..., -0.5600,  1.2421,  1.5442],
         [-1.8123, -2.1959,  0.6806,  ..., -1.0523, -1.6041, -0.8999]]])

In [93]:
x=b

In [63]:
x = b
x.data["input_values"] = x.data["input_values"].squeeze()
if "attention_mask" in x.data:
    x.data["attention_mask"] = x.data["attention_mask"].squeeze()

In [94]:
model(x.input_values, x.attention_mask).last_hidden_state.shape

/home/jacopo/PycharmProjects/progetto-tesi/.venv/lib/python3.12/site-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


torch.Size([1, 49, 768])

In [58]:
o = x.data["input_values"].unbind(0)